In [2]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import pickle
from  ModuleNo7My import Numerical_Inversion
from  ModuleNo7My import make_tr_va_te
from  ModuleNo7My import month_range

import ipynb_path
now_file_name = ipynb_path.get().split('/')[-1].replace('.ipynb','')

input_path = '../input/'
status_file_name = 'status.csv'
station_file_name = 'station.csv'
trip_file_name = 'trip.csv'
weather_file_name = 'weather.csv'
trip_start_date_file_name = 'trip_start_date.csv'
trip_end_date_file_name = 'trip_end_date.csv'
train_sta_wea_tri_end_file_name = 'train_all_row.csv'
train_sta_wea_notna_all_file_name = 'all_row_notna.csv'

output_path = '../output/'
model_path = '../model/'

In [3]:
# #実行したらコメントアウトする
# #保存ファイルの重複が起こる可能性がある
# ###########################################################
# # もう一度notebookを動かす際はモデルリストのstart_numとend_numを
# # 設定してください
# ##########################################################
# st_in = input('初めの数字を入力してください')
# end_in = input('終わりの数字を入力してください')
# start_num = int(st_in)
# end_num = int(end_in)

# model_name_list = [f'model_{i}_{now_file_name}.sav' for i in range(start_num, end_num)]
# submit_file_name_list = [f'submission_{i}_{now_file_name}.csv' for i in range(start_num,end_num)]
# model_submit_dict = dict(zip(model_name_list,submit_file_name_list))

初めの数字を入力してください 5
終わりの数字を入力してください 20


In [27]:
train_sta_wea = pd.read_csv(input_path + train_sta_wea_notna_all_file_name)

In [28]:
not_use_col_list = ['id', 'month', 'day', 'hour', 'bikes_available','predict', 'date' ]
NI = Numerical_Inversion(train_sta_wea,not_use_col_list)

In [5]:
df_ss = NI.make_converted_df('SS')
df_mm = NI.make_converted_df('MM')
df_rs = NI.make_converted_df('RS')
df_log = NI.make_converted_df('LOG')
df_yeo = NI.make_converted_df('YEO')

In [22]:
df_mm = NI.make_converted_df('MM')

In [21]:
df_mm_log = NI_log.make_converted_df('MM')

In [42]:
df_lag_in = train_sta_wea.copy()
df_lag_in['bike_lag_1'] = df_lag_in.groupby('station_id')['bikes_available'].shift(1).fillna(method='bfill')
df_lag_in['bike_lag_2'] = df_lag_in.groupby('station_id')['bikes_available'].shift(2).fillna(method='bfill')
df_lag_in['bike_lag_3'] = df_lag_in.groupby('station_id')['bikes_available'].shift(3).fillna(method='bfill')

# df_lag_in['bike_lag_1'] = df_lag_in.groupby('station_id')['bikes_available'].shift(1).fillna(method='bfill')
# df_lag_in['bike_lag_2'] = df_lag_in.groupby('station_id')['bikes_available'].shift(2).fillna(method='bfill')
# df_lag_in['bike_lag_3'] = df_lag_in.groupby('station_id')['bikes_available'].shift(3).fillna(method='bfill')

In [30]:
df_lag_in.isnull().sum()

id                              0
year                            0
month                           0
day                             0
hour                            0
station_id                      0
bikes_available            416405
predict                         0
date                            0
max_temperature                 0
mean_temperature                0
min_temperature                 0
max_dew_point                   0
mean_dew_point                  0
min_dew_point                   0
max_humidity                    0
mean_humidity                   0
min_humidity                    0
max_sea_level_pressure          0
mean_sea_level_pressure         0
min_sea_level_pressure          0
max_visibility                  0
mean_visibility                 0
min_visibility                  0
max_wind_Speed                  0
mean_wind_speed                 0
precipitation                   0
cloud_cover                     0
wind_dir_degrees                0
lat           

In [34]:
NI_log = Numerical_Inversion(df_lag_in,not_use_col_list)

df_yeo = NI_log.make_converted_df('YEO')

df_yeo_only = NI_log.convert('YEO')

df_yeo_not_convert = df_lag_in[not_use_col_list]

In [35]:
#SVD（特異値分解）を使う

def make_SVD(df,num):
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=num, random_state=84)
    svd.fit(df)
    all_yeo_svd = svd.transform(df)
    all_yeo_svd_df = pd.DataFrame(df, columns=['svd_1', 'svd_2', 'svd_3', 'svd_4', 'svd_5'])
    
    return all_yeo_svd_df

In [39]:
df_yeo_only

,year,station_id,max_temperature,mean_temperature,min_temperature,max_dew_point,mean_dew_point,min_dew_point,max_humidity,mean_humidity,...,events_Rain,events_なし,city_city1,city_city2,city_city3,city_city4,city_city5,bike_lag_1,bike_lag_2,bike_lag_3
0,0.000000,0.000000,0.565762,1.002574,1.552186,2.956244,2.721465,2.060178,1.147425,0.940755,...,-0.0,205.709949,0.167618,0.0,-0.0,-0.0,-0.0,0.298937,0.298944,0.298951
1,0.000000,0.000000,0.565762,1.002574,1.552186,2.956244,2.721465,2.060178,1.147425,0.940755,...,-0.0,205.709949,0.167618,0.0,-0.0,-0.0,-0.0,0.298937,0.298944,0.298951
2,0.000000,0.000000,0.565762,1.002574,1.552186,2.956244,2.721465,2.060178,1.147425,0.940755,...,-0.0,205.709949,0.167618,0.0,-0.0,-0.0,-0.0,0.298937,0.298944,0.298951
3,0.000000,0.000000,0.565762,1.002574,1.552186,2.956244,2.721465,2.060178,1.147425,0.940755,...,-0.0,205.709949,0.167618,0.0,-0.0,-0.0,-0.0,0.298937,0.298944,0.298951
4,0.000000,0.000000,0.565762,1.002574,1.552186,2.956244,2.721465,2.060178,1.147425,0.940755,...,-0.0,205.709949,0.167618,0.0,-0.0,-0.0,-0.0,0.298937,0.298944,0.298951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226395,1.144033,0.908601,0.579810,1.083634,1.552186,2.956244,2.482704,1.908759,1.323734,0.811056,...,-0.0,205.709949,0.167618,0.0,-0.0,-0.0,-0.0,0.210479,0.210482,0.210485
1226396,1.144033,0.908601,0.579810,1.083634,1.552186,2.956244,2.482704,1.908759,1.323734,0.811056,...,-0.0,205.709949,0.167618,0.0,-0.0,-0.0,-0.0,0.234298,0.210482,0.210485
1226397,1.144033,0.908601,0.579810,1.083634,1.552186,2.956244,2.482704,1.908759,1.323734,0.811056,...,-0.0,205.709949,0.167618,0.0,-0.0,-0.0,-0.0,0.234298,0.234302,0.210485
1226398,1.144033,0.908601,0.579810,1.083634,1.552186,2.956244,2.482704,1.908759,1.323734,0.811056,...,-0.0,205.709949,0.167618,0.0,-0.0,-0.0,-0.0,0.234298,0.234302,0.234306


In [36]:
yeo_svd_df = make_SVD(df_yeo_only, 5)

In [48]:

df_lag_mm = NI_log.make_converted_df('MM')

In [23]:
all_svd_log = pd.concat([df_mm_log, yeo_svd_df])

In [26]:
all_svd_log.isnull().sum()

year                1226400
station_id          1226400
max_temperature     1226400
mean_temperature    1226400
min_temperature     1226400
                     ...   
svd_1               2452800
svd_2               2452800
svd_3               2452800
svd_4               2452800
svd_5               2452800
Length: 62, dtype: int64

In [46]:
# #保存する　df_lag_inを
# df_lag_in.to_csv(input_path + 'all_lag_in.csv')

In [ ]:
#保存する　all_svd_logを
#all_svd_log.to_csv(input_path + 'all_svd_log.csv')

In [ ]:
all_yeo_s = pd.concat([all_yeo, all_yeo_svd_df], axis=1)

In [47]:
df_lag_in 

,id,year,month,day,hour,station_id,bikes_available,predict,date,max_temperature,...,events_Rain,events_なし,city_city1,city_city2,city_city3,city_city4,city_city5,bike_lag_1,bike_lag_2,bike_lag_3
0,0,2013,9,1,0,0,11.0,0,2013-09-01,81,...,0,1,1,0,0,0,0,11.0,11.0,11.0
1,1,2013,9,1,1,0,11.0,0,2013-09-01,81,...,0,1,1,0,0,0,0,11.0,11.0,11.0
2,2,2013,9,1,2,0,11.0,0,2013-09-01,81,...,0,1,1,0,0,0,0,11.0,11.0,11.0
3,3,2013,9,1,3,0,11.0,0,2013-09-01,81,...,0,1,1,0,0,0,0,11.0,11.0,11.0
4,4,2013,9,1,4,0,11.0,0,2013-09-01,81,...,0,1,1,0,0,0,0,11.0,11.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226395,1226395,2015,8,31,19,69,8.0,0,2015-08-31,82,...,0,1,1,0,0,0,0,7.0,7.0,7.0
1226396,1226396,2015,8,31,20,69,8.0,0,2015-08-31,82,...,0,1,1,0,0,0,0,8.0,7.0,7.0
1226397,1226397,2015,8,31,21,69,8.0,0,2015-08-31,82,...,0,1,1,0,0,0,0,8.0,8.0,7.0
1226398,1226398,2015,8,31,22,69,8.0,0,2015-08-31,82,...,0,1,1,0,0,0,0,8.0,8.0,8.0


In [11]:
# df_
# df_bike_lag_2 = train_sta_wea['bikes_availabike_lag_1 = train_sta_wea['bikes_available'].shift(1)ble'].shift(2)
# df_bike_lag_3 = train_sta_wea['bikes_available'].shift(3)

In [12]:
df_lag_in = 
not_use_col_list = ['id', 'month', 'day', 'hour', 'bikes_available','predict', 'date' ]



In [ ]:
#SVD（特異値分解）を使う
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=5, random_state=84)
svd.fit(all_X_yeo_float)
all_yeo_svd = svd.transform(all_X_yeo_float)
all_yeo_svd_df = pd.DataFrame(all_yeo_svd, columns=['svd_1', 'svd_2', 'svd_3', 'svd_4', 'svd_5'])
all_yeo_s = pd.concat([all_yeo, all_yeo_svd_df], axis=1)



In [8]:
# from datetime import date
# from datetime import datetime as dt
# from dateutil.relativedelta import relativedelta
# sub_df_all = pd.DataFrame()
# valid_score_list = []

# #分析のメインになるdfを設定する
# main_df = df_ss.copy()

# #train_end_next_dateにはvalidationの一ヶ月の初めを入れる
# for d in month_range(dt(2014,8,1), dt(2015,8,1)):
#     make_data = make_tr_va_te(main_df, d)
#     train_notna = make_data.make_train_data()
#     valid_notna = make_data.make_valid_data()
#     test =make_data.make_test_data()
#     tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
#     te_X = make_data.predict_for_data(test)
#     model = make_data.make_fit_model(tr_X, tr_y, va_X, va_y)
#     model_name = model_name_list[0]
#     sub_df, valid_best_score = make_data.predict(model_path, model_name)
#     sub_df_all = pd.concat([sub_df_all, sub_df])
#     valid_score_list.append(valid_best_score)
# cv_score = sum(valid_score_list)/len(valid_score_list)
# print('***')
# print('***')
# print(f'CV score is {cv_score}')
# print(f'model_name is {model_name}')
# model_name_list.pop(0)

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035744 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.17966	valid_1's rmse: 3.39808
[200]	training's rmse: 3.04801	valid_1's rmse: 3.27533
[300]	training's rmse: 2.99494	valid_1's rmse: 3.22871
[400]	training's rmse: 2.96376	valid_1's rmse: 3.20423
[500]	training's rmse: 2.94062	valid_1's rmse: 3.19084
[600]	training's rmse: 2.92298	valid_1's rmse: 3.18297
[700]	training's rmse: 2.90758	valid_1's rmse: 3.17775
[800]	training's rmse: 2.89209	valid_1's rmse: 3.17459
[900]	training's rmse: 2.87939	valid_1's rmse: 3.17095
[1000]	training's rmse: 2.8675	valid_1's rmse: 3.16876
[1100]	training's rmse: 2.85652	valid_1's rmse: 3.16589
[1200]	training's rmse: 2.84626	valid_1's rmse: 3.16439
[1300]	training's rmse: 2.8362	valid_1's rmse: 

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035555 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.17966	valid_1's rmse: 3.39808
[200]	training's rmse: 3.04801	valid_1's rmse: 3.27533
[300]	training's rmse: 2.99494	valid_1's rmse: 3.22871
[400]	training's rmse: 2.96376	valid_1's rmse: 3.20423
[500]	training's rmse: 2.94062	valid_1's rmse: 3.19084
[600]	training's rmse: 2.92298	valid_1's rmse: 3.18297
[700]	training's rmse: 2.90758	valid_1's rmse: 3.17775
[800]	training's rmse: 2.89209	valid_1's rmse: 3.17459
[900]	training's rmse: 2.87939	valid_1's rmse: 3.17095
[1000]	training's rmse: 2.8675	valid_1's rmse: 3.16876
[1100]	training's rmse: 2.85652	valid_1's rmse: 3.16589
[1200]	training's rmse: 2.84626	valid_1's rmse: 3.16439
[1300]	training's rmse: 2.8362	valid_1's rmse: 

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


...

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.054269 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.33273	valid_1's rmse: 3.49688
[200]	training's rmse: 3.20418	valid_1's rmse: 3.41845
[300]	training's rmse: 3.14878	valid_1's rmse: 3.38508
[400]	training's rmse: 3.11415	valid_1's rmse: 3.36075
[500]	training's rmse: 3.09033	valid_1's rmse: 3.3469
[600]	training's rmse: 3.07043	valid_1's rmse: 3.34009
[700]	training's rmse: 3.05457	valid_1's rmse: 3.33661
[800]	training's rmse: 3.04219	valid_1's rmse: 3.33437
[900]	training's rmse: 3.0309	valid_1's rmse: 3.33085
[1000]	training's rmse: 3.021	valid_1's rmse: 3.32978
[1100]	training's rmse: 3.01061	valid_1's rmse: 3.3287
[1200]	training's rmse: 3.00124	valid_1's rmse: 3.32675
[1300]	training's rmse: 2.99292	valid_1's rmse: 3.3

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050537 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.33273	valid_1's rmse: 3.49688
[200]	training's rmse: 3.20418	valid_1's rmse: 3.41845
[300]	training's rmse: 3.14878	valid_1's rmse: 3.38508
[400]	training's rmse: 3.11415	valid_1's rmse: 3.36075
[500]	training's rmse: 3.09033	valid_1's rmse: 3.3469
[600]	training's rmse: 3.07043	valid_1's rmse: 3.34009
[700]	training's rmse: 3.05457	valid_1's rmse: 3.33661
[800]	training's rmse: 3.04219	valid_1's rmse: 3.33437
[900]	training's rmse: 3.0309	valid_1's rmse: 3.33085
[1000]	training's rmse: 3.021	valid_1's rmse: 3.32978
[1100]	training's rmse: 3.01061	valid_1's rmse: 3.3287
[1200]	training's rmse: 3.00124	valid_1's rmse: 3.32675
[1300]	training's rmse: 2.99292	valid_1's rmse: 3.3

'model_0_No_7_my.sav'

In [11]:
# from datetime import date
# from datetime import datetime as dt
# from dateutil.relativedelta import relativedelta
# sub_df_all = pd.DataFrame()
# valid_score_list = []

# #分析のメインになるdfを設定する
# main_df = df_mm.copy()

# #train_end_next_dateにはvalidationの一ヶ月の初めを入れる
# for d in month_range(dt(2014,8,1), dt(2015,8,1)):
#     make_data = make_tr_va_te(main_df, d)
#     train_notna = make_data.make_train_data()
#     valid_notna = make_data.make_valid_data()
#     test =make_data.make_test_data()
#     tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
#     te_X = make_data.predict_for_data(test)
#     model = make_data.make_fit_model(tr_X, tr_y, va_X, va_y)
#     model_name = model_name_list[0]
#     sub_df, valid_best_score = make_data.predict(model_path, model_name)
#     sub_df_all = pd.concat([sub_df_all, sub_df])
#     valid_score_list.append(valid_best_score)
# cv_score = sum(valid_score_list)/len(valid_score_list)
# print('***')
# print('***')
# print(f'CV score is {cv_score}')
# print(f'model_name is {model_name}')
# model_name_list.pop(0)

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.033670 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.1801	valid_1's rmse: 3.39867
[200]	training's rmse: 3.04872	valid_1's rmse: 3.27589
[300]	training's rmse: 2.99601	valid_1's rmse: 3.22969
[400]	training's rmse: 2.96435	valid_1's rmse: 3.20438
[500]	training's rmse: 2.94185	valid_1's rmse: 3.19041
[600]	training's rmse: 2.92312	valid_1's rmse: 3.18143
[700]	training's rmse: 2.90798	valid_1's rmse: 3.17586
[800]	training's rmse: 2.8934	valid_1's rmse: 3.17047
[900]	training's rmse: 2.88022	valid_1's rmse: 3.16692
[1000]	training's rmse: 2.86945	valid_1's rmse: 3.16478
[1100]	training's rmse: 2.85951	valid_1's rmse: 3.16373
[1200]	training's rmse: 2.8494	valid_1's rmse: 3.16167
[1300]	training's rmse: 2.83984	valid_1's rmse: 3

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030723 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.1801	valid_1's rmse: 3.39867
[200]	training's rmse: 3.04872	valid_1's rmse: 3.27589
[300]	training's rmse: 2.99601	valid_1's rmse: 3.22969
[400]	training's rmse: 2.96435	valid_1's rmse: 3.20438
[500]	training's rmse: 2.94185	valid_1's rmse: 3.19041
[600]	training's rmse: 2.92312	valid_1's rmse: 3.18143
[700]	training's rmse: 2.90798	valid_1's rmse: 3.17586
[800]	training's rmse: 2.8934	valid_1's rmse: 3.17047
[900]	training's rmse: 2.88022	valid_1's rmse: 3.16692
[1000]	training's rmse: 2.86945	valid_1's rmse: 3.16478
[1100]	training's rmse: 2.85951	valid_1's rmse: 3.16373
[1200]	training's rmse: 2.8494	valid_1's rmse: 3.16167
[1300]	training's rmse: 2.83984	valid_1's rmse: 3

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


...

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038849 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.33272	valid_1's rmse: 3.49663
[200]	training's rmse: 3.20339	valid_1's rmse: 3.41385
[300]	training's rmse: 3.1481	valid_1's rmse: 3.37972
[400]	training's rmse: 3.11338	valid_1's rmse: 3.3551
[500]	training's rmse: 3.08944	valid_1's rmse: 3.3431
[600]	training's rmse: 3.07032	valid_1's rmse: 3.33678
[700]	training's rmse: 3.05446	valid_1's rmse: 3.33161
[800]	training's rmse: 3.04168	valid_1's rmse: 3.32971
[900]	training's rmse: 3.03088	valid_1's rmse: 3.32791
[1000]	training's rmse: 3.02012	valid_1's rmse: 3.3241
[1100]	training's rmse: 3.00963	valid_1's rmse: 3.32156
[1200]	training's rmse: 2.99963	valid_1's rmse: 3.31858
[1300]	training's rmse: 2.99128	valid_1's rmse: 3.

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035196 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.33272	valid_1's rmse: 3.49663
[200]	training's rmse: 3.20339	valid_1's rmse: 3.41385
[300]	training's rmse: 3.1481	valid_1's rmse: 3.37972
[400]	training's rmse: 3.11338	valid_1's rmse: 3.3551
[500]	training's rmse: 3.08944	valid_1's rmse: 3.3431
[600]	training's rmse: 3.07032	valid_1's rmse: 3.33678
[700]	training's rmse: 3.05446	valid_1's rmse: 3.33161
[800]	training's rmse: 3.04168	valid_1's rmse: 3.32971
[900]	training's rmse: 3.03088	valid_1's rmse: 3.32791
[1000]	training's rmse: 3.02012	valid_1's rmse: 3.3241
[1100]	training's rmse: 3.00963	valid_1's rmse: 3.32156
[1200]	training's rmse: 2.99963	valid_1's rmse: 3.31858
[1300]	training's rmse: 2.99128	valid_1's rmse: 3.

'model_1_No_7_my.sav'

In [14]:
# from datetime import date
# from datetime import datetime as dt
# from dateutil.relativedelta import relativedelta
# sub_df_all = pd.DataFrame()
# valid_score_list = []

# #分析のメインになるdfを設定する
# main_df = df_rs.copy()

# #train_end_next_dateにはvalidationの一ヶ月の初めを入れる
# for d in month_range(dt(2014,8,1), dt(2015,8,1)):
#     make_data = make_tr_va_te(main_df, d)
#     train_notna = make_data.make_train_data()
#     valid_notna = make_data.make_valid_data()
#     test =make_data.make_test_data()
#     tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
#     te_X = make_data.predict_for_data(test)
#     model = make_data.make_fit_model(tr_X, tr_y, va_X, va_y)
#     model_name = model_name_list[0]
#     sub_df, valid_best_score = make_data.predict(model_path, model_name)
#     sub_df_all = pd.concat([sub_df_all, sub_df])
#     valid_score_list.append(valid_best_score)
# cv_score = sum(valid_score_list)/len(valid_score_list)
# print('***')
# print('***')
# print(f'CV score is {cv_score}')
# print(f'model_name is {model_name}')
# model_name_list.pop(0)

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029128 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.1801	valid_1's rmse: 3.39867
[200]	training's rmse: 3.04872	valid_1's rmse: 3.27589
[300]	training's rmse: 2.99601	valid_1's rmse: 3.22969
[400]	training's rmse: 2.96435	valid_1's rmse: 3.20438
[500]	training's rmse: 2.94185	valid_1's rmse: 3.19041
[600]	training's rmse: 2.92312	valid_1's rmse: 3.18143
[700]	training's rmse: 2.90798	valid_1's rmse: 3.17586
[800]	training's rmse: 2.8934	valid_1's rmse: 3.17047
[900]	training's rmse: 2.88022	valid_1's rmse: 3.16692
[1000]	training's rmse: 2.86945	valid_1's rmse: 3.16478
[1100]	training's rmse: 2.85951	valid_1's rmse: 3.16373
[1200]	training's rmse: 2.8494	valid_1's rmse: 3.16167
[1300]	training's rmse: 2.83984	valid_1's rmse: 3

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026926 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.1801	valid_1's rmse: 3.39867
[200]	training's rmse: 3.04872	valid_1's rmse: 3.27589
[300]	training's rmse: 2.99601	valid_1's rmse: 3.22969
[400]	training's rmse: 2.96435	valid_1's rmse: 3.20438
[500]	training's rmse: 2.94185	valid_1's rmse: 3.19041
[600]	training's rmse: 2.92312	valid_1's rmse: 3.18143
[700]	training's rmse: 2.90798	valid_1's rmse: 3.17586
[800]	training's rmse: 2.8934	valid_1's rmse: 3.17047
[900]	training's rmse: 2.88022	valid_1's rmse: 3.16692
[1000]	training's rmse: 2.86945	valid_1's rmse: 3.16478
[1100]	training's rmse: 2.85951	valid_1's rmse: 3.16373
[1200]	training's rmse: 2.8494	valid_1's rmse: 3.16167
[1300]	training's rmse: 2.83984	valid_1's rmse: 3

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


...

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.042364 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.33236	valid_1's rmse: 3.4962
[200]	training's rmse: 3.20312	valid_1's rmse: 3.41312
[300]	training's rmse: 3.14741	valid_1's rmse: 3.3802
[400]	training's rmse: 3.11366	valid_1's rmse: 3.35796
[500]	training's rmse: 3.09	valid_1's rmse: 3.34499
[600]	training's rmse: 3.07101	valid_1's rmse: 3.33825
[700]	training's rmse: 3.05571	valid_1's rmse: 3.33376
[800]	training's rmse: 3.04278	valid_1's rmse: 3.33118
[900]	training's rmse: 3.03194	valid_1's rmse: 3.32941
[1000]	training's rmse: 3.02024	valid_1's rmse: 3.32598
[1100]	training's rmse: 3.00981	valid_1's rmse: 3.32424
[1200]	training's rmse: 3.00012	valid_1's rmse: 3.32179
[1300]	training's rmse: 2.99131	valid_1's rmse: 3.3

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.041327 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.33236	valid_1's rmse: 3.4962
[200]	training's rmse: 3.20312	valid_1's rmse: 3.41312
[300]	training's rmse: 3.14741	valid_1's rmse: 3.3802
[400]	training's rmse: 3.11366	valid_1's rmse: 3.35796
[500]	training's rmse: 3.09	valid_1's rmse: 3.34499
[600]	training's rmse: 3.07101	valid_1's rmse: 3.33825
[700]	training's rmse: 3.05571	valid_1's rmse: 3.33376
[800]	training's rmse: 3.04278	valid_1's rmse: 3.33118
[900]	training's rmse: 3.03194	valid_1's rmse: 3.32941
[1000]	training's rmse: 3.02024	valid_1's rmse: 3.32598
[1100]	training's rmse: 3.00981	valid_1's rmse: 3.32424
[1200]	training's rmse: 3.00012	valid_1's rmse: 3.32179
[1300]	training's rmse: 2.99131	valid_1's rmse: 3.3

'model_2_No_7_my.sav'

In [17]:
# from datetime import date
# from datetime import datetime as dt
# from dateutil.relativedelta import relativedelta
# sub_df_all = pd.DataFrame()
# valid_score_list = []

# #分析のメインになるdfを設定する
# main_df = df_log.copy()

# #train_end_next_dateにはvalidationの一ヶ月の初めを入れる
# for d in month_range(dt(2014,8,1), dt(2015,8,1)):
#     make_data = make_tr_va_te(main_df, d)
#     train_notna = make_data.make_train_data()
#     valid_notna = make_data.make_valid_data()
#     test =make_data.make_test_data()
#     tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
#     te_X = make_data.predict_for_data(test)
#     model = make_data.make_fit_model(tr_X, tr_y, va_X, va_y)
#     model_name = model_name_list[0]
#     sub_df, valid_best_score = make_data.predict(model_path, model_name)
#     sub_df_all = pd.concat([sub_df_all, sub_df])
#     valid_score_list.append(valid_best_score)
# cv_score = sum(valid_score_list)/len(valid_score_list)
# print('***')
# print('***')
# print(f'CV score is {cv_score}')
# print(f'model_name is {model_name}')
# model_name_list.pop(0)

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.025645 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.18639	valid_1's rmse: 3.40346
[200]	training's rmse: 3.05962	valid_1's rmse: 3.28799
[300]	training's rmse: 3.00412	valid_1's rmse: 3.24392
[400]	training's rmse: 2.97247	valid_1's rmse: 3.21629
[500]	training's rmse: 2.94898	valid_1's rmse: 3.19975
[600]	training's rmse: 2.92941	valid_1's rmse: 3.18877
[700]	training's rmse: 2.91332	valid_1's rmse: 3.18214
[800]	training's rmse: 2.89845	valid_1's rmse: 3.17791
[900]	training's rmse: 2.88493	valid_1's rmse: 3.17375
[1000]	training's rmse: 2.87396	valid_1's rmse: 3.1699
[1100]	training's rmse: 2.86346	valid_1's rmse: 3.16633
[1200]	training's rmse: 2.85479	valid_1's rmse: 3.16381
[1300]	training's rmse: 2.84625	valid_1's rmse:

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027643 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.18639	valid_1's rmse: 3.40346
[200]	training's rmse: 3.05962	valid_1's rmse: 3.28799
[300]	training's rmse: 3.00412	valid_1's rmse: 3.24392
[400]	training's rmse: 2.97247	valid_1's rmse: 3.21629
[500]	training's rmse: 2.94898	valid_1's rmse: 3.19975
[600]	training's rmse: 2.92941	valid_1's rmse: 3.18877
[700]	training's rmse: 2.91332	valid_1's rmse: 3.18214
[800]	training's rmse: 2.89845	valid_1's rmse: 3.17791
[900]	training's rmse: 2.88493	valid_1's rmse: 3.17375
[1000]	training's rmse: 2.87396	valid_1's rmse: 3.1699
[1100]	training's rmse: 2.86346	valid_1's rmse: 3.16633
[1200]	training's rmse: 2.85479	valid_1's rmse: 3.16381
[1300]	training's rmse: 2.84625	valid_1's rmse:

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


...

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035408 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.33845	valid_1's rmse: 3.50484
[200]	training's rmse: 3.21113	valid_1's rmse: 3.42077
[300]	training's rmse: 3.15818	valid_1's rmse: 3.39248
[400]	training's rmse: 3.12296	valid_1's rmse: 3.37526
[500]	training's rmse: 3.09945	valid_1's rmse: 3.36717
[600]	training's rmse: 3.08076	valid_1's rmse: 3.35605
[700]	training's rmse: 3.06619	valid_1's rmse: 3.34844
[800]	training's rmse: 3.05193	valid_1's rmse: 3.34416
[900]	training's rmse: 3.03817	valid_1's rmse: 3.34021
[1000]	training's rmse: 3.0271	valid_1's rmse: 3.3385
[1100]	training's rmse: 3.01738	valid_1's rmse: 3.34059
Early stopping, best iteration is:
[1031]	training's rmse: 3.0241	valid_1's rmse: 3.33778


/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039705 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.33845	valid_1's rmse: 3.50484
[200]	training's rmse: 3.21113	valid_1's rmse: 3.42077
[300]	training's rmse: 3.15818	valid_1's rmse: 3.39248
[400]	training's rmse: 3.12296	valid_1's rmse: 3.37526
[500]	training's rmse: 3.09945	valid_1's rmse: 3.36717
[600]	training's rmse: 3.08076	valid_1's rmse: 3.35605
[700]	training's rmse: 3.06619	valid_1's rmse: 3.34844
[800]	training's rmse: 3.05193	valid_1's rmse: 3.34416
[900]	training's rmse: 3.03817	valid_1's rmse: 3.34021
[1000]	training's rmse: 3.0271	valid_1's rmse: 3.3385
[1100]	training's rmse: 3.01738	valid_1's rmse: 3.34059
Early stopping, best iteration is:
[1031]	training's rmse: 3.0241	valid_1's rmse: 3.33778
*****
3.337777

'model_3_No_7_my.sav'

In [49]:
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
sub_df_all = pd.DataFrame()
valid_score_list = []

#分析のメインになるdfを設定する
main_df = df_lag_mm.copy()

#train_end_next_dateにはvalidationの一ヶ月の初めを入れる
for d in month_range(dt(2014,8,1), dt(2015,8,1)):
    make_data = make_tr_va_te(main_df, d)
    train_notna = make_data.make_train_data()
    valid_notna = make_data.make_valid_data()
    test =make_data.make_test_data()
    tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
    te_X = make_data.predict_for_data(test)
    model = make_data.make_fit_model(tr_X, tr_y, va_X, va_y)
    model_name = model_name_list[0]
    sub_df, valid_best_score = make_data.predict(model_path, model_name)
    sub_df_all = pd.concat([sub_df_all, sub_df])
    valid_score_list.append(valid_best_score)
cv_score = sum(valid_score_list)/len(valid_score_list)
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')
model_name_list.pop(0)

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037612 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.88058	valid_1's rmse: 2.02455
[200]	training's rmse: 1.41148	valid_1's rmse: 1.561
[300]	training's rmse: 1.30487	valid_1's rmse: 1.45961
[400]	training's rmse: 1.25699	valid_1's rmse: 1.41673
[500]	training's rmse: 1.23043	valid_1's rmse: 1.39627
[600]	training's rmse: 1.21549	valid_1's rmse: 1.38591
[700]	training's rmse: 1.20602	valid_1's rmse: 1.38006
[800]	training's rmse: 1.19882	valid_1's rmse: 1.37572
[900]	training's rmse: 1.19209	valid_1's rmse: 1.3716
[1000]	training's rmse: 1.18607	valid_1's rmse: 1.36898
[1100]	training's rmse: 1.18149	valid_1's rmse: 1.36763
[1200]	training's rmse: 1.17729	valid_1's rmse: 1.36622
[1300]	training's rmse: 1.17325	valid_1's rmse: 1

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027568 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.88058	valid_1's rmse: 2.02455
[200]	training's rmse: 1.41148	valid_1's rmse: 1.561
[300]	training's rmse: 1.30487	valid_1's rmse: 1.45961
[400]	training's rmse: 1.25699	valid_1's rmse: 1.41673
[500]	training's rmse: 1.23043	valid_1's rmse: 1.39627
[600]	training's rmse: 1.21549	valid_1's rmse: 1.38591
[700]	training's rmse: 1.20602	valid_1's rmse: 1.38006
[800]	training's rmse: 1.19882	valid_1's rmse: 1.37572
[900]	training's rmse: 1.19209	valid_1's rmse: 1.3716
[1000]	training's rmse: 1.18607	valid_1's rmse: 1.36898
[1100]	training's rmse: 1.18149	valid_1's rmse: 1.36763
[1200]	training's rmse: 1.17729	valid_1's rmse: 1.36622
[1300]	training's rmse: 1.17325	valid_1's rmse: 1

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


...

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039161 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.95292	valid_1's rmse: 2.02613
[200]	training's rmse: 1.46588	valid_1's rmse: 1.56615
[300]	training's rmse: 1.3549	valid_1's rmse: 1.46114
[400]	training's rmse: 1.3087	valid_1's rmse: 1.41225
[500]	training's rmse: 1.28341	valid_1's rmse: 1.38573
[600]	training's rmse: 1.26991	valid_1's rmse: 1.37406
[700]	training's rmse: 1.25961	valid_1's rmse: 1.36635
[800]	training's rmse: 1.25209	valid_1's rmse: 1.36099
[900]	training's rmse: 1.24554	valid_1's rmse: 1.35756
[1000]	training's rmse: 1.24005	valid_1's rmse: 1.35483
[1100]	training's rmse: 1.23556	valid_1's rmse: 1.35222
[1200]	training's rmse: 1.23145	valid_1's rmse: 1.35051
[1300]	training's rmse: 1.22769	valid_1's rmse: 

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038542 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.95292	valid_1's rmse: 2.02613
[200]	training's rmse: 1.46588	valid_1's rmse: 1.56615
[300]	training's rmse: 1.3549	valid_1's rmse: 1.46114
[400]	training's rmse: 1.3087	valid_1's rmse: 1.41225
[500]	training's rmse: 1.28341	valid_1's rmse: 1.38573
[600]	training's rmse: 1.26991	valid_1's rmse: 1.37406
[700]	training's rmse: 1.25961	valid_1's rmse: 1.36635
[800]	training's rmse: 1.25209	valid_1's rmse: 1.36099
[900]	training's rmse: 1.24554	valid_1's rmse: 1.35756
[1000]	training's rmse: 1.24005	valid_1's rmse: 1.35483
[1100]	training's rmse: 1.23556	valid_1's rmse: 1.35222
[1200]	training's rmse: 1.23145	valid_1's rmse: 1.35051
[1300]	training's rmse: 1.22769	valid_1's rmse: 

'model_5_No_7_my.sav'

In [50]:
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')

***
***
CV score is 1.3268642699651376
model_name is model_5_No_7_my.sav


In [51]:
# submit_file_name = model_submit_dict[model_name]
# sub_df_all.to_csv(output_path+submit_file_name, index=False, header=False)
# print(submit_file_name)

submission_5_No_7_my.csv


In [53]:
# ! signate submit --competition-id=550 '/Users/ken/OneDrive - Yokohama City University/活動/data-sci/データ分析遊び/competition/SIGNATE/オペレーション最適化に向けたシェアサイクルの利用予測/output/submission_5_No_7_my.csv'



You have successfully submitted your predictions.We will send you the submission result to your email address.


In [52]:
#特徴量重要度を算出する
cols = list(tr_X.columns)
f_importance = np.array(model.feature_importance())
# f_importance = f_importance / np.sum(f_importance)

df_importance = pd.DataFrame({'feature': cols, 'importance':f_importance})
df_importance = df_importance.sort_values('importance', ascending=False)

display(df_importance)

,feature,importance
47,bike_lag_1,10100
52,hour,7254
26,start_date,6343
1,station_id,5505
29,end_date,5448
28,end_duration,5058
22,lat,4951
25,start_duration,4438
23,long,4431
49,bike_lag_3,3358
